In [1]:
from edc import check_compatibility
check_compatibility("user-2023.07-01", dependencies=["GEODB"])



---------

The following environment variables are available:

* `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_DOMAIN`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_API_SERVER_PORT`


## Sharing Data

In [2]:
from xcube_geodb.core.geodb import GeoDBClient

In [3]:
geodb = GeoDBClient()
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

In [4]:
ds = geodb.get_my_collections()

if ds[(ds.database == geodb.whoami) & (ds.collection == 'land_use')].collection.count() == 0:
    # Have a look at fiona feature schema
    collections = {
            "land_use": 
            {
                "crs": 3794,
                "properties": 
                {
                    "RABA_PID": "float", 
                    "RABA_ID": "float", 
                    "D_OD": "date"
                }
            }
        }


    geodb.create_collections(collections)
    import geopandas
    import os
    gdf = geopandas.read_file("/shared/eurodatacube/curated/data/sample/land_use.shp")
    geodb.insert_into_collection('land_use', gdf)

### Publish a Collection to the World

In [5]:
geodb.list_my_grants()

,collection,grantee
0,2021-10-04-test-3,postgres
1,2021-10-04-test-3,postgres
2,2021-10-04-test-3,postgres
3,2021-10-04-test-3,postgres
4,2021-10-04-test-3,postgres
5,2021-10-04-test-3,postgres
6,2021-10-04-test-3,postgres
7,2021-10-04-test-1234,postgres
8,2021-10-04-test-1234,postgres
9,2021-10-04-test-1234,postgres


In [6]:
geodb.publish_collection("land_use")

See the chanhes made to the collection: 

In [7]:
geodb.get_event_log('land_use')

,event_type,message,username,date
0,added rows,9827 rows inserted into geodb_418dfeac-15f0-46...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:03.607382
1,dropped rows,from collection geodb_418dfeac-15f0-4606-9edb-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:04.126797
2,added property,"{name: test_prop, type: integer} to collection...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:06.172545
3,dropped property,test_prop from collection geodb_418dfeac-15f0-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.069597
4,added property,"{name: test1, type: integer} to collection geo...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.337766
...,...,...,...,...
216,dropped property,test2 from collection geodb_418dfeac-15f0-4606...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2023-07-31T10:08:15.762979
217,dropped,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2023-07-31T10:08:16.554061
218,created,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2023-07-31T10:09:26.168786
219,added rows,9827 rows inserted into geodb_418dfeac-15f0-46...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2023-07-31T10:09:30.684603


### Accessing Collection as a different User

Please be aware that you cannot change the user easily within a Notebook. Here, we changed the credentials in the background. The following cells will not run and are for demonstration only.

In [8]:
geodb = GeoDBClient()
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

### Revoke access

In [9]:
geodb.list_my_grants()

,collection,grantee
0,2021-10-04-test-3,postgres
1,2021-10-04-test-3,postgres
2,2021-10-04-test-3,postgres
3,2021-10-04-test-3,postgres
4,2021-10-04-test-3,postgres
5,2021-10-04-test-3,postgres
6,2021-10-04-test-3,postgres
7,2021-10-04-test-1234,postgres
8,2021-10-04-test-1234,postgres
9,2021-10-04-test-1234,postgres


In [10]:
geodb.unpublish_collection("land_use")

See the chanhes made to the collection: 

In [11]:
geodb.get_event_log('land_use')

,event_type,message,username,date
0,added rows,9827 rows inserted into geodb_418dfeac-15f0-46...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:03.607382
1,dropped rows,from collection geodb_418dfeac-15f0-4606-9edb-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:04.126797
2,added property,"{name: test_prop, type: integer} to collection...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:06.172545
3,dropped property,test_prop from collection geodb_418dfeac-15f0-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.069597
4,added property,"{name: test1, type: integer} to collection geo...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.337766
...,...,...,...,...
217,dropped,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2023-07-31T10:08:16.554061
218,created,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2023-07-31T10:09:26.168786
219,added rows,9827 rows inserted into geodb_418dfeac-15f0-46...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2023-07-31T10:09:30.684603
220,published,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2023-07-31T10:10:58.599296


In [12]:
geodb.list_my_grants()

,collection,grantee
0,2021-10-04-test-3,postgres
1,2021-10-04-test-3,postgres
2,2021-10-04-test-3,postgres
3,2021-10-04-test-3,postgres
4,2021-10-04-test-3,postgres
5,2021-10-04-test-3,postgres
6,2021-10-04-test-3,postgres
7,2021-10-04-test-1234,postgres
8,2021-10-04-test-1234,postgres
9,2021-10-04-test-1234,postgres


In [13]:
gdf = geodb.get_collection_by_bbox(collection="land_use", bbox=(452750.0, 88909.549, 464000.0, 102486.299),
                comparison_mode="contains", bbox_crs=3794, limit=2, offset=10)
gdf

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,11,2023-07-31T10:09:28.69554+00:00,None,"POLYGON ((460137.998 95628.898, 460111.001 956...",5983161,1100,2019-03-11
1,12,2023-07-31T10:09:28.69554+00:00,None,"POLYGON ((453673.609 91328.224, 453678.929 913...",5983074,1600,2019-03-26
